In [19]:
import scda
import pprint
import logging
import os
import numpy as np
scda.configure_log()
import pprint

# Small survey demo to run on NCCS Discover

In [20]:
survey_params = {'Pupil': { 'prim': ['hex3', 'hex4'], 'gap':1, 'edge':'round',
                            'secobs': 'X', 'thick': '025',
                            'centobs': [False, True], 'N': 250 },
                 'FPM': { 'rad': [3.5, 4.0], 'M':60 },
                 'LS': { 'shape':'ann', 'obscure':0, 'aligntol':None, 
                         'id':[20, 25], 'od':[81] },
                 'Image': { 'ida':-0.5, 'bw':0.10, 'Nlam':3}}

In [21]:
survey_dir = os.path.normpath("/astro/opticslab1/SCDA/Scripts/AMPL/survey_demo_july2016")
if not os.path.exists(survey_dir):
    os.mkdir(survey_dir)
    print("Created survey directory {:s}".format(survey_dir))
else:
    print("The survey directory {:s} already exists".format(survey_dir))

os.chdir(survey_dir)
survey_fname = os.path.basename(survey_dir)
ampl_src_dir = os.path.normpath("./amplsrc")
slurm_dir = os.path.normpath("./slurmsh")
sol_dir = os.path.normpath("./solutions")
log_dir = os.path.normpath("./logs")
TelAp_dir = os.path.normpath("../InputMasks/TelAp")
LS_dir = os.path.normpath("../InputMasks/LS")
FPM_dir = os.path.normpath("../InputMasks/FPM")

fileorg = {'work dir':'.', 'ampl src dir':ampl_src_dir, 'slurm dir':slurm_dir,
           'log dir':log_dir, 'sol dir':sol_dir,
           'TelAp dir':TelAp_dir, 'LS dir':LS_dir, 'FPM dir':FPM_dir}

The survey directory /astro/opticslab1/SCDA/Scripts/AMPL/survey_demo_july2016 already exists


In [22]:
fileorg

{'FPM dir': '../InputMasks/FPM',
 'LS dir': '../InputMasks/LS',
 'TelAp dir': '../InputMasks/TelAp',
 'ampl src dir': 'amplsrc',
 'log dir': 'logs',
 'slurm dir': 'slurmsh',
 'sol dir': 'solutions',
 'work dir': '.'}

In [23]:
os.listdir(fileorg['LS dir'])

['LS_quart_ann20D70_clear_N0250.dat',
 'LS_quart_ann20D80_clear_N0250.dat',
 'LS_quart_ann20D90_clear_N0250.dat',
 'LS_quart_ann20D81_clear_N0250.dat',
 'LDZ_quart_ann20D81_clear_Tol02_N0250.dat',
 'LS_quart_ann25D81_clear_N0250.dat',
 'LDZ_quart_ann25D81_clear_Tol02_N0250.dat']

## Initiate a survey object with the above parameter combinations 

In [24]:
small_survey = scda.DesignParamSurvey(scda.QuarterplaneAPLC, survey_params, fileorg=fileorg)
print("This survey has {0:d} design parameter combinations.".format(small_survey.N_combos))
print("{0:d} parameters are varied: {1}".format(len(small_survey.varied_param_index), small_survey.varied_param_index))

This survey has 16 design parameter combinations.
5 parameters are varied: (('Pupil', 'prim'), ('Pupil', 'centobs'), ('FPM', 'rad'), ('LS', 'id'), ('LS', 'od'))


In [25]:
os.getcwd()

'/astro/opticslab1/SCDA/Scripts/AMPL/survey_demo_july2016'

## Show the file organization scheme

In [26]:
pprint.pprint(small_survey.fileorg)

{'FPM dir': '../InputMasks/FPM',
 'LS dir': '../InputMasks/LS',
 'TelAp dir': '../InputMasks/TelAp',
 'ampl src dir': 'amplsrc',
 'eval dir': '.',
 'log dir': 'logs',
 'slurm dir': 'slurmsh',
 'sol dir': 'solutions',
 'work dir': '.'}


## Show some attributes of an individual coronagraph object

In [27]:
#i = 100
i = 0
print("Telescope aperture file for design #{:d}: {:s}".format(i+1, small_survey.coron_list[i].fileorg['TelAp fname']))
print("Focal plane mask file for design #{:d}: {:s}".format(i+1, small_survey.coron_list[i].fileorg['FPM fname']))
print("Lyot stop file for design #{:d}: {:s}".format(i+1, small_survey.coron_list[i].fileorg['LS fname']))

Telescope aperture file for design #1: ../InputMasks/TelAp/TelAp_quart_hex3X025cobs0gap1_N0250.dat
Focal plane mask file for design #1: ../InputMasks/FPM/FPM_quart_occspot_M060.dat
Lyot stop file for design #1: ../InputMasks/LS/LS_quart_ann20D81_clear_N0250.dat


### Check the status of input files needed to run the AMPL program

In [28]:
small_survey.check_ampl_input_files()

True

## List the varying parameter combinations

In [29]:
pprint.pprint(small_survey.varied_param_combos)

(('hex3', False, 3.5, 20, 81),
 ('hex3', False, 3.5, 25, 81),
 ('hex3', False, 4.0, 20, 81),
 ('hex3', False, 4.0, 25, 81),
 ('hex3', True, 3.5, 20, 81),
 ('hex3', True, 3.5, 25, 81),
 ('hex3', True, 4.0, 20, 81),
 ('hex3', True, 4.0, 25, 81),
 ('hex4', False, 3.5, 20, 81),
 ('hex4', False, 3.5, 25, 81),
 ('hex4', False, 4.0, 20, 81),
 ('hex4', False, 4.0, 25, 81),
 ('hex4', True, 3.5, 20, 81),
 ('hex4', True, 3.5, 25, 81),
 ('hex4', True, 4.0, 20, 81),
 ('hex4', True, 4.0, 25, 81))


## Write the batch of AMPL files

In [30]:
small_survey.write_ampl_batch(override_infile_status=False, overwrite=True)

INFO:root:Wrote all 16 of 16 design survey AMPL programs into amplsrc


## Write the batch of queue execution scripts

In [31]:
small_survey.write_slurm_batch(overwrite=True, queue_spec='12h')

INFO:root:Wrote all 16 of 16 design survey slurm scripts into slurmsh


## Write tables summarizing the design survey configuration and status to a spreadsheet

In [32]:
small_survey.write_spreadsheet()

INFO:root:Wrote design survey spreadsheet to ./survey_demo_july2016_ntz_2016-07-26.csv


## Store the design survey as a serialized python object

In [33]:
small_survey.write()

INFO:root:Wrote the design parameter survey object to ./survey_demo_july2016_ntz_2016-07-26.pkl


## Load an existing design survey

In [34]:
os.listdir('.')

['amplsrc',
 'solutions',
 'logs',
 'slurmsh',
 'survey_demo_july2016_ntz_2016-07-26.csv',
 'survey_demo_july2016_ntz_2016-07-26.pkl']

In [35]:
mysurvey = scda.load_design_param_survey(small_survey.fileorg['survey fname'])